# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
assert len(song_files) > 0, "No song files found!"

In [6]:
song_files[0]

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

## #1: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Convert the array to a list and set it to `artist_data`

In [7]:
artist_columns=['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
artist_data = pd.DataFrame()
for f in song_files:
    tmp_df=pd.read_json(f, typ='series')[artist_columns]
    artist_data = artist_data.append(tmp_df, ignore_index=True)
artist_data.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual
1,ARNTLGG11E2835DDB9,NaN,,NaN,Clp
2,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project
3,AR10USD1187B99F3F1,NaN,"Burlington, Ontario, Canada",NaN,Tweeterfriendly Music
4,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops


#### Insert records into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [8]:
print(artist_table_insert)
print(artist_data.shape[0])


INSERT INTO artists (artist_id, name, location, latitude, longitude)
VALUES (%s, %s, %s, %s, %s)

82


In [9]:
artist_data=artist_data.rename(columns={
    "artist_name":"name"
    , "artist_location":"location"
    , "artist_latitude":"latitude"
    , "artist_longitude":"longitude"})
artist_data.head(1)

,artist_id,latitude,location,longitude,name
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual


In [10]:
artist_data.shape[0]

82

In [11]:
artist_data=artist_data[['artist_id', 'name', 'location', 'latitude', 'longitude']].drop_duplicates()

In [12]:
artist_data.shape[0]

69

In [13]:
artist_data.to_csv('artist_data.csv', index=False, header = False)

In [14]:
sql="""COPY artists FROM '/home/workspace/artist_data.csv'  
(DELIMITER(','), FORMAT csv)"""
#, FORCE_NOT_NULL(latitude,longitude))
print(sql)
cur.execute(sql)
conn.commit()

COPY artists FROM '/home/workspace/artist_data.csv'  
(DELIMITER(','), FORMAT csv)


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [15]:
song_columns=['song_id', 'title', 'artist_id', 'year','duration']
# song_data = pd.read_json(song_files[1], typ='series')[song_columns].values.tolist()
# song_data

# artist_columns=['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
song_data = pd.DataFrame()
for f in song_files:
    tmp_df=pd.read_json(f, typ='series')[song_columns]
    song_data = song_data.append(tmp_df, ignore_index=True)
song_data.head(1)

,artist_id,duration,song_id,title,year
0,ARD7TVE1187B99BFB1,218.93179,SOMZWCG12A8C13C480,I Didn't Mean To,0.0


In [16]:
song_data.shape[0]

82

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [17]:
song_data.head(1)

,artist_id,duration,song_id,title,year
0,ARD7TVE1187B99BFB1,218.93179,SOMZWCG12A8C13C480,I Didn't Mean To,0.0


In [18]:
song_data['year']=song_data['year'].astype(int).astype(str)
song_data=song_data[['song_id', 'title', 'artist_id', 'year', 'duration']].drop_duplicates()


In [19]:
song_data.head(1)
song_data.shape[0]

71

In [20]:
# conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
# cur = conn.cursor()
# cur.execute("Select * from artists")
# results = cur.fetchmany(5)
# results

In [21]:
song_data.to_csv('song_data.csv', index=False, header = False)
# conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
# cur = conn.cursor()
sql="""COPY songs FROM '/home/workspace/song_data.csv'  
(DELIMITER(','), FORMAT csv)"""
print(sql)
cur.execute(sql)
conn.commit()


COPY songs FROM '/home/workspace/song_data.csv'  
(DELIMITER(','), FORMAT csv)


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [22]:
log_files = get_files('data/log_data')

In [23]:
assert len(log_files) > 0, "No log files found!"

In [24]:
#logfile_columns=['artist', 'auth', 'firstName', 'gender', 'itemInSession']
log_df = pd.DataFrame()
for f in log_files:
    tmp_df=pd.read_json(f, lines=True)
    log_df = log_df.append(tmp_df, ignore_index=True)
log_df.head()


# log_df =pd.read_json(log_files[1], lines=True)
# log_df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [25]:
log_df = log_df[(log_df.page == 'NextSong')]


In [26]:
# Cribbed from https://stackoverflow.com/questions/34883101/pandas-converting-row-with-unix-timestamp-in-milliseconds-to-datetime
#df.ts = pd.to_datetime(df['ts'], unit='ms')
log_df['ts'] = pd.to_datetime(log_df['ts'], unit='ms')
#df.loc[row_indexer,col_indexer] =pd.to_datetime(df['ts'], unit='ms')
log_df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [27]:
time_df = log_df.ts.to_frame().drop_duplicates()
time_df['hour']=log_df.ts.dt.hour
time_df['day']=log_df.ts.dt.day
time_df['week_of_year']= log_df.ts.dt.weekofyear
time_df['month']=log_df.ts.dt.month
time_df['year']=log_df.ts.dt.year
time_df['weekday']=log_df.ts.dt.dayofweek # Monday=0; Sunday=6
time_df.head(1)

,ts,hour,day,week_of_year,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [28]:
time_df.to_csv('time_data.csv', index=False, header = False)

In [29]:

sql="""COPY time FROM '/home/workspace/time_data.csv'  
(DELIMITER(','), FORMAT csv)"""
print(sql)
cur.execute(sql)
conn.commit()


COPY time FROM '/home/workspace/time_data.csv'  
(DELIMITER(','), FORMAT csv)


In [30]:
# for i, row in time_df.iterrows():
#     cur.execute(time_table_insert, list(row))
#     conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [31]:
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level','ts']]


In [1]:
user_df=user_df.sort_values(by='ts')
# user_df=user_df.set_index('userId')

user_df.userId=user_df.userId.astype('int')

NameError: name 'user_df' is not defined

In [33]:
user_df[(user_df['userId']==75)]

,userId,firstName,lastName,gender,level,ts
6443,75,Joseph,Gutierrez,M,free,2018-11-02 16:18:50.796
6444,75,Joseph,Gutierrez,M,free,2018-11-02 16:22:13.796
6445,75,Joseph,Gutierrez,M,free,2018-11-02 16:24:54.796
6446,75,Joseph,Gutierrez,M,free,2018-11-02 16:27:56.796
6447,75,Joseph,Gutierrez,M,free,2018-11-02 16:30:54.796
6450,75,Joseph,Gutierrez,M,free,2018-11-02 16:34:47.796
6452,75,Joseph,Gutierrez,M,free,2018-11-02 16:38:55.796
6454,75,Joseph,Gutierrez,M,free,2018-11-02 16:46:02.796
6455,75,Joseph,Gutierrez,M,free,2018-11-02 16:49:44.796
6456,75,Joseph,Gutierrez,M,free,2018-11-02 16:51:37.796


In [34]:

user_df=user_df.drop_duplicates(subset=['userId'],keep='last')
user_df[(user_df['userId']==75)]

,userId,firstName,lastName,gender,level,ts
7423,75,Joseph,Gutierrez,M,free,2018-11-29 17:49:47.796


In [39]:
user_df[(user_df['userId']==7)]

,userId,firstName,lastName,gender,level
7584,7,Adelyn,Jordan,F,free


In [38]:
user_df.shape[0]


96

In [38]:
user_df.shape[0]

96

In [39]:
#user_df = user_df.drop_duplicates()

In [40]:
user_df.shape[0]

96

In [35]:
user_df=user_df.drop(columns=['ts'])

In [42]:
user_df.duplicated()

6475    False
6730    False
6094    False
6296    False
3705    False
3767    False
3935    False
3957    False
7949    False
7584    False
7783    False
7813    False
7876    False
1209    False
1271    False
1372    False
1374    False
1429    False
1440    False
1455    False
1522    False
4510    False
529     False
530     False
538     False
781     False
791     False
881     False
899     False
928     False
        ...  
7423    False
7443    False
7495    False
7541    False
7554    False
7565    False
7581    False
7582    False
19      False
27      False
56      False
62      False
105     False
125     False
161     False
177     False
186     False
194     False
219     False
220     False
252     False
271     False
282     False
288     False
308     False
325     False
370     False
381     False
385     False
387     False
Length: 96, dtype: bool

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [36]:
user_df.to_csv('user_data.csv', header=False, index=False)

In [37]:

conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

sql="""
COPY users FROM '/home/workspace/user_data.csv'  
(DELIMITER(','), FORMAT csv)
"""
print(sql)
cur.execute(sql)
conn.commit()


COPY users FROM '/home/workspace/user_data.csv'  
(DELIMITER(','), FORMAT csv)



In [45]:
# for i, row in user_df.iterrows():
#     cur.execute(user_table_insert, row)
#     conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [46]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
sql="""
select song_id, songs.artist_id, artists.name, songs.title, songs.duration
FROM songs 
INNER JOIN artists 
ON songs.artist_id = artists.artist_id
"""
cur.execute(sql)
data=cur.fetchall()
#sond_detail_df=

In [47]:
# From: https://www.linkedin.com/pulse/how-create-pandas-data-frame-postgresql-psycopg-vitor-spadotto
cols=[]
for col in cur.description:
    cols.append(col[0])
song_details=pd.DataFrame(data=data, columns=cols)


In [48]:
song_details.duration=song_details.duration.astype(float)
song_details=song_details.rename(columns={'name':'artist','duration':'length','title':'song'})


In [49]:
song_details.artist=song_details.artist.str.strip()



In [50]:
df.artist=df.artist.str.strip()

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [51]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [52]:
song_plays = df.merge(song_details, how='inner', on=['artist','song','length'])
# songplay_id SERIAL PRIMARY KEY
# , start_time timestamp REFERENCES time
# , user_id int REFERENCES users
# , level varchar
# , song_id varchar 
# , artist_id varchar REFERENCES artists
# , session_id int
# , location varchar
# , user_agent varchar


In [53]:
song_plays=song_plays[['ts','userId','level','song_id','artist_id','sessionId','location','userAgent']]

In [54]:
song_plays.to_csv('song_play_data.csv', header=False, index=False)

In [56]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
sql="""

CREATE TABLE songplays_tmp as (SELECT * FROM songplays) with no data;

COPY songplays (
start_time
, user_id
, level
, song_id
, artist_id
, session_id
, location
, user_agent) FROM '/home/workspace/song_play_data.csv'  
(DELIMITER(','), FORMAT csv);

INSERT INTO songplays 
SELECT * FROM songplays_tmp;

DROP TABLE songplays_tmp;
"""
sql="""

COPY songplays (
start_time
, user_id
, level
, song_id
, artist_id
, session_id
, location
, user_agent) FROM '/home/workspace/song_play_data.csv'  
(DELIMITER(','), FORMAT csv);

"""


#sql="COPY songplays FROM '/home/workspace/song_play_data.csv'  (DELIMITER(','), FORMAT csv);"


print(sql)
cur.execute(sql)
conn.commit()



COPY songplays (
start_time
, user_id
, level
, song_id
, artist_id
, session_id
, location
, user_agent) FROM '/home/workspace/song_play_data.csv'  
(DELIMITER(','), FORMAT csv);




IntegrityError: duplicate key value violates unique constraint "no_dups"
DETAIL:  Key (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)=(2018-11-21 21:56:47.796, 15, paid, SOZCTXZ12AB0182364, AR5KOSW1187FB35FF4, 818, Chicago-Naperville-Elgin, IL-IN-WI, "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36") already exists.
CONTEXT:  COPY songplays, line 1


In [180]:
song_plays.head()

,ts,userId,level,song_id,artist_id,sessionId,location,userAgent
0,2018-11-21 21:56:47.796,15,paid,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 0 to 8055
Data columns (total 18 columns):
artist           6820 non-null object
auth             6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
method           6820 non-null object
page             6820 non-null object
registration     6820 non-null float64
sessionId        6820 non-null int64
song             6820 non-null object
status           6820 non-null int64
ts               6820 non-null datetime64[ns]
userAgent        6820 non-null object
userId           6820 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(12)
memory usage: 1.3+ MB


In [121]:
song_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 5 columns):
song_id      71 non-null object
artist_id    71 non-null object
artist       71 non-null object
song         71 non-null object
length       71 non-null float64
dtypes: float64(1), object(4)
memory usage: 2.9+ KB


### Try the recommended approach (iterative insert)

In [122]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

df.sort_values(by='artist')

for index, row in df.iterrows():

    song_select="SELECT song_id, songs.artist_id FROM songs INNER JOIN artists ON songs.artist_id = artists.artist_id WHERE name = '%s' AND title = '%s'"
    # Attempting to handle ' in artist/song names
    # Using here - but vulnerable to sql injection - see here
    #https://stackoverflow.com/questions/12316953/insert-text-with-single-quotes-in-postgresql
    song=row.song.replace('\'', '\'\'')
    artist=row.artist.replace('\'', '\'\'')
    sql=song_select % (artist,song)
#    print(sql)
#     song_select % (song, artist)
#    print(song_select)
#    print(row.song)
#    print()
#    print(where % (row.song, row.artist))#row.artist, row.song)
    #print(song_select)
    #, row.length)
    # get songid and artistid from song and artist tables
    cur.execute(sql)
    results = cur.fetchone()
#    print(results)
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
songplay_table_insert="""
INSERT INTO songplays (
songplay_id
, start_time
, user_id
, level
, song_id
, artist_id
, session_id
, location
, user_agent)
VALUES (%s, %s, %s, %s, %s)
"""
        
        
    # insert songplay record
#     songplay_data = ()
#     cur.execute(songplay_table_insert, songplay_data)
#     conn.commit()

In [123]:
cur.execute(song_select+where)
results = cur.fetchall()
results

NameError: name 'where' is not defined

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.